In [3]:
from itertools import cycle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import lasso_path
from sklearn import datasets
from sklearn import linear_model
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import tqdm
import cvxpy as cp
import numpy as np


def compute_q(p):
    if p != np.inf and p > 1:
        q = p / (p - 1)
    elif p == 1:
        q = np.inf
    else:
        q = 1
    return q


class AdversarialTraining:
    def __init__(self, X, y, p):
        m, n = X.shape
        q = compute_q(p)
        # Formulate problem
        param = cp.Variable(n)
        param_norm = cp.pnorm(param, p=q)
        adv_radius = cp.Parameter(name='adv_radius', nonneg=True)
        abs_error = cp.abs(X @ param - y)
        adv_loss = 1 / m * cp.sum((abs_error + adv_radius * param_norm) ** 2)
        prob = cp.Problem(cp.Minimize(adv_loss))
        self.prob = prob
        self.adv_radius = adv_radius
        self.param = param
        self.warm_start = False

    def __call__(self, adv_radius, **kwargs):
        try:
            self.adv_radius.value = adv_radius
            self.prob.solve(warm_start=self.warm_start, **kwargs)
            v = self.param.value
        except:
            v = np.zeros(self.param.shape)
        return v


def get_lasso_path(X, y, eps_lasso=1e-2):
    alphas, coefs, _ = lasso_path(X, y, eps=eps_lasso)
    coefs= np.concatenate([np.zeros([X.shape[1], 1]), coefs], axis=1)
    alphas = np.concatenate([1e2 * np.ones([1]), alphas], axis=0)
    return alphas, coefs, []


def get_path(X, y, estimator, amax, eps=1e-5, n_alphas=200):
    amin = eps * amax
    alphas = np.logspace(np.log10(amin), np.log10(amax), n_alphas)
    coefs_ = []
    for a in tqdm.tqdm(alphas):
        coefs = estimator(X, y, a)
        coefs_.append(coefs if coefs is not None else np.zeros(m))
    return alphas, np.stack((coefs_)).T


def plot_coefs(alphas, coefs, ax):
    colors = cycle(["b", "r", "g", "c", "k"])
    for coef_l, c in zip(coefs, colors):
        ax.semilogx(1/alphas, coef_l, c=c)


def plot_coefs_l1norm(coefs, ax):
    colors = cycle(["b", "r", "g", "c", "k"])
    l1norm = np.abs(coefs).mean(axis=0)
    for coef_l, c in zip(coefs, colors):
        ax.plot(l1norm, coef_l, c=c)


def multiple_imputation(nbr_mi, X_nan):
    for i in range(nbr_mi):
       n_i = np.random.randint(0, 1000)
       ice = IterativeImputer()
       ice_mean = IterativeImputer(random_state=n_i, max_iter=50, sample_posterior=True)
       res = ice_mean.fit_transform(X_nan)
       #print("fin res ", res)
       return res

#X, y = datasets.load_diabetes(return_X_y=True)
#print(X.shape, y.shape)
# Standardize data

n = 200
d = 4
X = np.random.rand(n, d)
X -= X.mean(axis=0)
X /= X.std(axis=0)
b = np.random.rand(d)
y = X @ b

masks = np.random.binomial(1, 0.1, (n, d))  # 1 missing, 0 seen
#M = np.sum(masks, axis=1)
#X_nan = X.copy()
#X_nan[masks == 1] = np.nan

#X = multiple_imputation(1, X_nan)
print("end block")


end block


In [ ]:
fig, ax = plt.subplots(num='advtrain_linf')
linfadvtrain = AdversarialTraining(X, y, p=np.inf)
estimator = lambda X, y, a:  linfadvtrain(adv_radius=a)
alphas_adv, coefs_advtrain_linf  = get_path(X, y, estimator, 1e1)
plot_coefs_l1norm(coefs_advtrain_linf, ax)

In [ ]:
fig, ax = plt.subplots(num='lasso')
alphas_lasso, coefs_lasso, _ = get_lasso_path(X, y)
plot_coefs_l1norm(coefs_lasso, ax)

In [50]:
# imputation given mean and covariance matrix

def imputation_elliptic(mu, sigma, x, masks):
  # mu, mean elliptical distribution (,d)
  # sigma, cov matrix elliptical distribution (d, d)
  # x: dataset (n, d)
  # masks: mask data, 0 seen, 1 missing
  n, d = x.shape
  print(n, d)
  x_imp = x.copy()
  for i in range(n):
    x_c = x[i, :]
    m_bool = (m[i, :] == 0)  # True seen, False missing
    sigma_aa_inv = np.linalg.inv(sigma[m_bool, :][:, m_bool])
    sigma_ma = sigma[~m_bool, :][:, m_bool]
    print(sigma_aa_inv)
    print(sigma_ma)
    print("wee", x_c[~m_bool])
    mu_cond = mu[~m_bool] + sigma_ma @ sigma_aa_inv @ (x_c[m_bool] - mu[m_bool])
    print(mu_cond)
    x_imp[i, ~m_bool] = mu_cond
  print("x_orig \n", x)
  print("x_imp \n", x_imp)
  print("diff\n", x - x_imp)
  return x_imp

n = 2
r = 4
S = np.random.randint(low=1, high=10, size=(r, r))
S = S.T @ S + np.eye(r)
print(S)
muu = np.array([1, 2, 3, 4])
x = np.random.randint(low=1, high=10, size=(n, d))
m = np.array([[1, 1, 1, 1], [0, 1, 1, 1]])

res = imputation_elliptic(muu, S, x, m)
#print(res)


[[229. 234. 124. 210.]
 [234. 244. 131. 217.]
 [124. 131. 103. 123.]
 [210. 217. 123. 203.]]
2 4
[]
[]
[]
wee [2 4 8 5]
[1. 2. 3. 4.]
[[0.00436681]]
[[234.]
 [124.]
 [210.]]
[[234.]
 [124.]
 [210.]]
wee [3 9 8]
[4.04366812 4.08296943 5.83406114]
x_orig 
 [[2 4 8 5]
 [3 3 9 8]]
x_imp 
 [[1 2 3 4]
 [3 4 4 5]]
diff
 [[ 1  2  5  1]
 [ 0 -1  5  3]]


In [27]:
r = 4
S = np.random.randint(low=1, high=4, size=(r, r))
S = S.T @ S + np.eye(r)
print(S)
m = np.array([0, 0, 1, 0])  # 1 missing, 0 seen
m_bool = (m == 0)
print(m_bool)
S_sub = S[m_bool, :][:, m_bool]
print(S_sub)

S_ma = S[m_bool, :][:, ~m_bool]
print(S_ma)

s = np.array([1, 2, 3, 4])
ss = s
print(s[m_bool])
print()

[[19. 12. 12. 16.]
 [12. 11. 10. 10.]
 [12. 10. 16. 10.]
 [16. 10. 10. 16.]]
[ True  True False  True]
[[19. 12. 16.]
 [12. 11. 10.]
 [16. 10. 16.]]
[[12.]
 [10.]
 [10.]]
[1 2 4]
